In [1]:
import pyrealsense2 as rs
import matplotlib as plt
import cv2
import logging
import numpy as np
import time

In [2]:
ctx = rs.context()
devices = ctx.query_devices()

In [3]:
var = [i.get_info(rs.camera_info.serial_number) for i in devices]

In [4]:
var

['923322071362', '923322070538']

In [7]:
#configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_device(var[0])
config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 90)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 60)
config.enable_record_to_file('simplecam.bag')
#start stream
pipeline.start(config)

try:
    start = time.time()
    while time.time() - start <10:
        
        frames = pipeline.wait_for_frames()
        depthframe = frames.get_depth_frame()
        colframe = frames.get_color_frame()
        if not depthframe:
            continue
        
        #depth_counter = depthframe.get_frame_metadata(rs.frame_metadata_value.frame_counter)
        #if depth_counter > 5400:
         #   break
        
        #convert images to numpy arrays
        depthim = np.asanyarray(depthframe.get_data())
        colim = np.asanyarray(colframe.get_data())
        
        #apply color map on depth image, convert to 8bit pixel
        depthcolmap = cv2.applyColorMap(cv2.convertScaleAbs(depthim, alpha=0.5), cv2.COLORMAP_JET)
        
        #stack all images horizontally
        images = np.hstack((colim, depthcolmap))
        
        #show images from cameras
        cv2.namedWindow('RealSense', cv2.WINDOW_NORMAL)
        cv2.imshow('RealSense', images)
        cv2.waitKey(1)
    
finally:
    #stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    del pipeline
    del config